In [2]:
#importing required packages
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from numpy import arange
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


In [3]:
patient_phy_data = pd.read_csv("../data/structured_info.csv")
patient_phy_data.drop(['Unnamed: 0'], axis = 1, inplace=True)
print(patient_phy_data.shape)
patient_phy_data.head()

(447057, 192)


,ins_medicare,ins_medicaid,ins_privatepay,ins_commercial,ins_other,memb_1yrprior,memb_at_dx,dodi,chances_of_recur,return_visit,...,patient_gender_Male,physician_gender_Female,physician_gender_Male,IS_TOBACCO_USER_Never,IS_TOBACCO_USER_Passive,IS_TOBACCO_USER_Quit,IS_TOBACCO_USER_Yes,HISPANIC_N,HISPANIC_U,HISPANIC_Y
0,1,0,1,0,0,1.0,1.0,1.0,0,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,0,1,0,0,1.0,1.0,1.0,0,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1,0,1,0,0,1.0,1.0,1.0,0,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1,0,1,0,0,1.0,1.0,1.0,0,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1,0,1,0,0,1.0,1.0,1.0,0,1,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [5]:
#pd.set_option('display.max_columns', None)
#patient_phy_data.describe()


In [4]:
X = patient_phy_data.drop('return_visit', axis=1)  # Replace 'target_variable_column_name' with your actual target variable column name
y = patient_phy_data['return_visit']

# Split the data into training (80%) and a test set (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [5]:
# List of variable counts to consider
variable_counts = [5, 6, 7, 10, 15, 20]

#Cross-validation
model_cv = LogisticRegressionCV(Cs=[0.001, 0.01, 0.1, 1, 10], cv=5, solver='liblinear')

# Create a LogisticRegressionCV model
# Cs: List of values for the regularization parameter C to be searched
# cv: Number of cross-validation folds
# solver: Algorithm to use in the optimization problem

model_cv.fit(X_train,y_train)

# Get the best C value found during cross-validation
best_C = model_cv.C_[0]

# Predict on the test set
predictions = model_cv.predict(X_test)

# Evaluate the model performance
accuracy = model_cv.score(X_test, y_test)

# Access the coefficients of the logistic regression model
coefficients = model_cv.coef_[0]

# Create a DataFrame to associate coefficients with feature names
feature_names = X.columns  # Assuming your input data is a DataFrame
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the DataFrame by absolute coefficient values to identify important features
sorted_coefficients_df = coefficients_df.reindex(coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)


for count in variable_counts:
    # Select the top 'count' variables
    selected_variables = sorted_coefficients_df.head(count)['Feature'].values
    X_subset = X_train[selected_variables]
    
    # Fit the model on the subset of variables
    model_cv.fit(X_subset, y_train)
    
    # Predict on the test set
    predictions = model_cv.predict(X_test[selected_variables])
    
    # Evaluate the model performance
    accuracy = accuracy_score(y_test, predictions)
    
    # Report the accuracy for the current model
    print(f'Top {count} variables - Accuracy: {accuracy}')

# Set the option to display all rows and columns of the DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)




Top 5 variables - Accuracy: 0.7153849595132644
Top 6 variables - Accuracy: 0.7153849595132644
Top 7 variables - Accuracy: 0.7153625911510759
Top 10 variables - Accuracy: 0.7198027110454972
Top 15 variables - Accuracy: 0.7212342862255625
Top 20 variables - Accuracy: 0.7218717845479354
